In [1]:
# imports
import pymongo
import datetime
import pandas as pd
from bson.json_util import dumps

# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.se_db

# Declare the collections
players = db.players_db
nonplayers = db.nonplayers_db
games = db.games_db

### Pull data

In [2]:
player_path = "./datacsvs/CleanPlayerData.csv"
playerdata = pd.read_csv(player_path)
realdata = playerdata[playerdata['Alignment'] != '0'] # eventually change to playerdata

nonplayer_path = "./datacsvs/NonPlayerData.csv"
npdata = pd.read_csv(nonplayer_path)

gamepath = './datacsvs/CleanGameData.csv'
game_data = pd.read_csv(gamepath)

# need original data for full list of unique players (thanks Tempus & Rubix)
# TODO: do the player ID stuff at the transform stage, not in this file
fullpath = './datacsvs/DataTest.csv' 
fulldata = pd.read_csv(fullpath)

### Set up player & game IDs

In [3]:
playerlist = fulldata['Player'].unique()
player_ids = {p:(i+1) for i,p in enumerate(playerlist)}

gamelist = realdata['Game'].unique()
game_ids = []
for i, g in enumerate(gamelist):
    if type(g) == str:
        curr_game = {
            'str':g,
            'id':i,
            'format':g[0:2],
            'num':g[2:]
        }
        game_ids.append(curr_game)

game_ids_df = pd.DataFrame(game_ids) # TODO again, do this before making CleanGameData

### Players (data test) into mongo

In [4]:
players.delete_many({}) # clear any stored data

stats_list = []
for p in realdata.iterrows():
    row = p[1] # p[0] is the index of the row, p[1] is the content


    # trying to get the game ID (can get grumpy with data mismatches)
    try: 
        curr_id = int(game_ids_df[game_ids_df['str'] == row['Game']].index[0])
    except:
        print(row)
        
    pgstat = {
        'player_id': player_ids.get(row['person'], 0),
        'player_name': row['person'],
        'game_id': curr_id,
        'game_str': row['Game'],
        'alignment': row['Alignment'],
        'fOutcome': row['fOutcome'],       
    }

    # Fill out fields below only if not "null"
    if type(row['Death']) != str:
        pgstat['death'] = '0'
    else:
        pgstat['death'] = row['Death']
    
    if row['HitNum'] != '-': # this is one way to deal with blank hitnums but is it a good way?
        pgstat['HitNum'] = row['HitNum']
        
        if row['Hit1'] != '-' :
            pgstat['Hit1'] = row['Hit1']
            pgstat['HitL'] = row['HitL']
            
    if row['Inactive'] == 'Y':
        pgstat['inactive'] = True
    else:
        pgstat['inactive'] = False
        
    if row['Role'] != '-':
        pgstat['role'] = row['Role']
    
    
    # stats_list.append(pgstat) # stores same as players db in a list, for ease of testing/debugging
    players.insert_one(pgstat)

In [5]:
# turn mongo db to json and write to json file
json_player_data = dumps(list(players.find()))
 
with open('./datajsons/playerdata.json', 'w') as file: 
    file.write(json_player_data) 

In [6]:
# # Verify results (that players is populated accurately):
# results = players.find()
# for result in results:
#     print(result)

### Nonplayers (data test) into mongo

In [7]:
# TODO: turn the below stuff into a function? probably easier with cleaner incoming data

nonplayers.delete_many({})

# @TODO: rename row2 and currid2 to more descriptive vars
np_stats = []
for n in npdata.iterrows():
    row2 = n[1]

    try: 
        curr_id2 = int(game_ids_df[game_ids_df['str'] == row2['Game']].index[0])
    except:
        print(row)
        
    npgstat = {
        'player_id': player_ids.get(row2['person'], 1000),
        'player_name': row2['person'],
        'game_id': curr_id2,
        'game_str': row2['Game'],
                
    }
            
    if row2['Spec'] == 'Y':
        npgstat['GM'] = False
        npgstat['spec'] = True
        npgstat['IM'] = False
    elif row2['GM'] == 'Y':
        npgstat['GM'] = True
        npgstat['spec'] = False
        npgstat['IM'] = False
    else:
        npgstat['GM'] = False
        npgstat['spec'] = False
        npgstat['IM'] = True

    
    #np_stats.append(npgstat) # for testing & debugging
    nonplayers.insert_one(npgstat)
    

In [8]:
# results = nonplayers.find()
# for result in results:
#     print(result)

In [9]:
# write to nonplayer json
json_nonplayer_data = dumps(list(nonplayers.find()))
with open('./datajsons/nonplayerdata.json', 'w') as file2: 
    file2.write(json_nonplayer_data) 

### Game data (game stats) into mongo

In [10]:

# grouping player data by game ID to get aggregate data
players_with_ids = realdata.merge(game_ids_df, left_on='Game', right_on='str')
grouped_pd = players_with_ids.groupby('id', sort = False)

In [11]:
# num players
broken_games = grouped_pd['Broken'].value_counts() # Why broken???Each game is either broken or not? this seems like a bad idea
game_ids_df['num_players'] = list(broken_games)

In [12]:
# alignment, faction outcome, death, inactivity, broken - from PlayerData
alignment_counts = grouped_pd['Alignment'].value_counts()
foutcome_counts = grouped_pd['fOutcome'].value_counts()
death_counts = grouped_pd['Death'].value_counts()
inactive_counts = grouped_pd['Inactive'].value_counts()
broken_counts = grouped_pd['Broken'].value_counts() # and if used broken above why here? what does this do

# uhh feel like this should be a dict
counts = [alignment_counts, foutcome_counts, death_counts, inactive_counts, broken_counts]
count_names = ['_align', '_win', '_death', '_inactive', '_broken']

# take each column and merge into game df
# (e.g. alignment column might have row {'Village': 30, 'Eliminator': 5})
for i, count in enumerate(counts): 
    #print(count_names[i]) # debugging
    count_df = count.unstack().fillna(0) # TODO: tell me what unstack() does, my previous self
    game_ids_df = game_ids_df.merge(count_df, how='left', left_index=True, right_index=True, suffixes = ('', count_names[i]))



_align
_win
_death
_inactive
_broken


In [14]:
# Setting, complexity, # of cycles - from GameData file
relevant_gd = game_data[['Setting', 'Game Complexity', '# of Cycles', 'Winning Team']]
relevant_gd = relevant_gd.fillna(0)
# relevant_gd = relevant_gd[relevant_gd['Setting'] != 0] # relies on apparent fact that setting is always filled in when any of them are

game_ids_df = game_ids_df.merge(relevant_gd, how='left', left_index=True, right_index=True)

In [15]:
game_ids_df = game_ids_df.fillna(0)

In [16]:
# Mod, GM(s), Spec(s) - from Nonplayer data

# RESULT: [GMs], Mod, [Specs]

mods = [0] * len(game_ids_df)
specs = [[] for i in range(len(game_ids_df))]
gms = [[] for i in range(len(game_ids_df))]

# loop through nonplayers list and put each cell into new field for correct game ID
for n in np_stats:
#     game_id = n['game_id']
#     place = specs[game_id]
    if n['IM']:
        mods[n['game_id']] = n['player_id']
    elif n['spec']:
        specs[n['game_id']].append(n['player_id'])
    elif n['GM']:
        gms[n['game_id']].append(n['player_id'])
    else:
        print(f"Problem: {n}") 

# to be honest this may be easier than method for players? 

In [17]:
# GAME DATA INTO MONGO

games.delete_many({})

game_stats = []

# rename certain fields so no identicaly letters
# (really with this format why letters? Either descriptive name or ID number reference)
align_list = ['G', 'E', 'B', 'F', 'D', 'M', 'N', 'C', 'S']
outcome_list = ['L', 'W', 'D_win']
death_list = ['E_death', 'L_death', 'V', 'S_death', 'F_death', 'N_death', 'M_death', 'I', 'D_death', 'O']

for n in game_ids_df.iterrows():
    item = n[1]
     
    game = item['id']

    # loops through columns list and makes dict of nums of each alignment-or-whatever
    aligns = {k:item[k] for k in align_list if item[k] > 0}
    outcomes = {k:item[k] for k in outcome_list if item[k] > 0}
    deaths = {k:item[k] for k in death_list if item[k] > 0}
        
        
    gstat = {

        'game_id': item['id'],
        'game_str': item['str'],
        'format': item['format'],
        'game_num': item['num'],
#         'broken': item['Broken'],   
        'num_players': item['num_players'],
        'alignment_counts': aligns,
        'outcome_counts': outcomes,
        'status_counts': deaths,
        'inactives': item['Y'],
        'broken': item['Y_broken'],
        'mod': mods[game],
        'spec': specs[game],
        'gm': gms[game],

        }
    
    # adds stuff from gamedata if it exists; a lot of it doesn't oops
    if (item['Winning Team'] != 0):
        gstat['winner'] = item['Winning Team']
    if (item['Winning Team'] != 0):
        gstat['setting'] = item['Setting']
    if (item['# of Cycles'] != 0):
        gstat['length'] = item['# of Cycles']
    if (item['Game Complexity'] != 0):
        gstat['complexity'] = item['Game Complexity']

            
                
    #game_stats.append(gstat) # for debugging
    
    games.insert_one(gstat)
           

In [18]:
# check for success
# results = games.find()
# for result in results:
#     print(result)

# print to json
json_game_data = dumps(list(games.find()))
with open('./datajsons/gamedata.json', 'w') as file3: 
    file3.write(json_game_data) 

In [19]:
# @TODO (game data)
# 'Eliminators Lynched', 'Eliminators Vig'd' - breakdown by alignment + death type?

# thread reply count
# start and end dates of game (or at least of thread, though aftermath would be way better - or at least second to last post to avoid closing post lateness)

# make things ints where relevant

# fix Broken (or wait to fix in source sheet)

In [29]:
# stats_list
# np_stats
# game_stats

# list of currently marked broken games
list(game_ids_df[game_ids_df['Y_broken'] > 0]['str'])

['LG3',
 'LG4',
 'MR1',
 'MR2',
 'LG8',
 'LG10',
 'QF6',
 'LG11',
 'MR6',
 'LG12',
 'MR7',
 'LG14',
 'LG15.1',
 'LG15.2',
 'MR10',
 'QF13',
 'LG18',
 'LG19',
 'LG20',
 'LG21',
 'MR14',
 'QF16',
 'MR15',
 'LG24',
 'LG26',
 'LG27',
 'LG29',
 'MR21',
 'LG33',
 'LG35',
 'MR23',
 'LG36',
 'LG38',
 'LG42',
 'LG43',
 'MR29',
 'LG46',
 'QF32',
 'QF33.1',
 'AG5',
 'QF40.1',
 'MR38.1',
 'LG61',
 'LG65']